# Day 13
## Puzzle 1

In [1]:
from collections import deque

In [112]:
def read_map(filename):
    """Return map"""
    with open(filename) as f:
        track = [i.replace('\n', '') for i in f.readlines()]
    return track

class Cart(object):
    """Cart object"""
    def __init__(self, x, y, symbol):
        self.x = x
        self.y = y
        self.initial_symbol = symbol
        self.angle = '>^<v'.index(symbol) * 90
        self.turns = deque([90, 0, -90])
        self.movement = {0: (1,0), 90: (0,-1), 180: (-1,0), 270: (0,1)}
        self.turn_dict = {}
        backslash_dict = {90: 90, 270: 90, 0: -90, 180: -90}
        forwardslash_dict = {90: -90, 270: -90, 0: 90, 180: 90}
        self.turn_dict['\\'] = backslash_dict
        self.turn_dict['/'] = forwardslash_dict
        
    def evolve(self, track):
        """Evolve one step on track"""
        step = self.movement[self.angle]
        newx = self.x + step[0]
        newy = self.y + step[1]
        assert newx >= 0
        assert newy >= 0
        newspace = track[newy][newx]
        if newspace == '+':
            addangle = self.turns[0]
            self.turns.rotate(-1)
        elif newspace == '|':
            assert self.angle in [90, 270]
            addangle = 0
        elif newspace == '-':
            assert self.angle in [0, 180]
            addangle = 0
        else:
            addangle = self.turn_dict[newspace][self.angle]
        self.angle = (self.angle + addangle) % 360
        self.x = newx
        self.y = newy
        
    def __repr__(self):
        return(str((self.x, self.y, self.angle)))

In [3]:
c = Cart(0,0,'>')
print(c)
c.evolve(['--\\'])
print(c)
c.evolve(['--\\'])
print(c)
c.evolve(['--\\'])
print(c)

(0, 0, 0)
(1, 0, 0)
(2, 0, 270)


IndexError: list index out of range

In [117]:
test_track = read_map("test_input.txt")
track = read_map("input.txt")

In [20]:
test_track

['/->-\\        ',
 '|   |  /----\\',
 '| /-+--+-\\  |',
 '| | |  | v  |',
 '\\-+-/  \\-+--/',
 '  \\------/   ']

In [147]:
def track2carts(track):
    carts = []
    for y, line in enumerate(track):
        for x, char in enumerate(line):
            if char in '>v<^':
                carts.append(Cart(x,y,char))
    return carts

def clean_track(track):
    cleantrack = []
    for line in track:
        s = line.replace(">", '-').replace("<", '-').replace("^", '|').replace("v", '|')
        cleantrack.append(s)
    return cleantrack

def check_collision(carts):
    coords = [(i.x, i.y) for i in carts]
    collision = False
    colcoords = (-1, -1)
    idxs = []
    for i, c in enumerate(coords):
        if coords.count(c) > 1:
            collision = True
            colcoords = c
            idxs = [i]
            for k, j in enumerate(coords):
                if j == c:
                    idxs.append(k)
            break
    return collision, colcoords, list(set(idxs))

In [114]:
carts = track2carts(test_track)
cleantrack = clean_track(test_track)
print(cleantrack)

['/---\\        ', '|   |  /----\\', '| /-+--+-\\  |', '| | |  | |  |', '\\-+-/  \\-+--/', '  \\------/   ']


In [115]:
print(carts)
for i in range(14):
    for j in carts:
        j.evolve(cleantrack)
    print(carts)

[(2, 0, 0), (9, 3, 270)]
[(3, 0, 0), (9, 4, 0)]
[(4, 0, 270), (10, 4, 0)]
[(4, 1, 270), (11, 4, 0)]
[(4, 2, 0), (12, 4, 90)]
[(5, 2, 0), (12, 3, 90)]
[(6, 2, 0), (12, 2, 90)]
[(7, 2, 0), (12, 1, 180)]
[(8, 2, 0), (11, 1, 180)]
[(9, 2, 270), (10, 1, 180)]
[(9, 3, 270), (9, 1, 180)]
[(9, 4, 180), (8, 1, 180)]
[(8, 4, 180), (7, 1, 270)]
[(7, 4, 90), (7, 2, 270)]
[(7, 3, 90), (7, 3, 270)]


In [131]:
check_collision(carts)

(False, (-1, -1), [])

In [148]:
carts = track2carts(track)
cleantrack = clean_track(track)
for i in range(1000):
    for j in carts:
        j.evolve(cleantrack)
        collision, coords, idxs = check_collision(carts)
        if collision:
            print(i, collision, coords, idxs)
            raise Exception

150 True (57, 104) [1, 14]


Exception: 

In [153]:
def evolve1(carts, track):
    remove_idx = []
    for i, cart in enumerate(carts):
        if i not in remove_idx:
            cart.evolve(track)
            collision, coords, idxs = check_collision(carts)
            if collision:
                for j in idxs:
                    if j not in remove_idx:
                        remove_idx.append(j)
    remove_idx.sort(reverse=True)
    if len(remove_idx) > 0: print(remove_idx)
    for i in remove_idx:
        carts.pop(i)
    return carts

In [154]:
track = read_map("input.txt")
carts = track2carts(track)
cleantrack = clean_track(track)
for i in range(100000):
    if i % 10000 == 0:
        print(i, len(carts))
    carts = evolve1(carts, cleantrack)
    if len(carts) == 1:
        print(carts)
        break

0 17
[14, 1]
[5, 1]
[9, 7]
[5, 4]
[8, 6]
[5, 1]
[3, 0]
10000 3
[1, 0]
[(67, 74, 180)]
